In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFNSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
FluxS = InterfaceData(size=inodes, value=0.0)
FluxF = InterfaceData(size=inodes, value=0.0)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 500
t = 0.0
avg_pc_iterations = 0.0
flog = open('FNSR%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FNSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # mapping
    FluxF.curr[:] = FluxS.curr[s2f]
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=FluxF.curr
    )
    
    while True:
        # back up previous interface value
        FluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid robin interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        FluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        FluxF.curr[:] = FluxS.curr[s2f]
        
        # update residual
        FluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(FluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(FluxF)
                under_relax.update_solution(FluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    
flog.close()

step=0, pc_iterations=21.
step=1, pc_iterations=21.
step=2, pc_iterations=19.
step=3, pc_iterations=20.
step=4, pc_iterations=17.
step=5, pc_iterations=19.
step=6, pc_iterations=16.
step=7, pc_iterations=18.
step=8, pc_iterations=17.
step=9, pc_iterations=17.
step=10, pc_iterations=17.
step=11, pc_iterations=16.
step=12, pc_iterations=17.
step=13, pc_iterations=17.
step=14, pc_iterations=14.
step=15, pc_iterations=16.
step=16, pc_iterations=14.
step=17, pc_iterations=16.
step=18, pc_iterations=14.
step=19, pc_iterations=16.
step=20, pc_iterations=14.
step=21, pc_iterations=14.
step=22, pc_iterations=14.
step=23, pc_iterations=14.
step=24, pc_iterations=14.
step=25, pc_iterations=14.
step=26, pc_iterations=14.
step=27, pc_iterations=14.
step=28, pc_iterations=14.
step=29, pc_iterations=14.
step=30, pc_iterations=14.
step=31, pc_iterations=14.
step=32, pc_iterations=14.
step=33, pc_iterations=14.
step=34, pc_iterations=13.
step=35, pc_iterations=14.
step=36, pc_iterations=13.
step=37, pc

step=297, pc_iterations=13.
step=298, pc_iterations=13.
step=299, pc_iterations=13.
step=300, pc_iterations=13.
step=301, pc_iterations=13.
step=302, pc_iterations=13.
step=303, pc_iterations=13.
step=304, pc_iterations=13.
step=305, pc_iterations=13.
step=306, pc_iterations=13.
step=307, pc_iterations=13.
step=308, pc_iterations=13.
step=309, pc_iterations=13.
step=310, pc_iterations=13.
step=311, pc_iterations=13.
step=312, pc_iterations=13.
step=313, pc_iterations=13.
step=314, pc_iterations=13.
step=315, pc_iterations=13.
step=316, pc_iterations=13.
step=317, pc_iterations=13.
step=318, pc_iterations=13.
step=319, pc_iterations=13.
step=320, pc_iterations=14.
step=321, pc_iterations=13.
step=322, pc_iterations=14.
step=323, pc_iterations=13.
step=324, pc_iterations=14.
step=325, pc_iterations=13.
step=326, pc_iterations=14.
step=327, pc_iterations=14.
step=328, pc_iterations=12.
step=329, pc_iterations=13.
step=330, pc_iterations=12.
step=331, pc_iterations=13.
step=332, pc_iterati

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFNSR%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.88749269 0.92855892 0.94089009 0.9403128  0.94148905 0.94384636
 0.94564009 0.9473331  0.94952839 0.95077429 0.95230707 0.95351001
 0.95504818 0.95620477 0.95750755 0.95849402 0.9597213  0.96074698
 0.96166413 0.96273019 0.96358231 0.96452155 0.96537666 0.96614673
 0.96699468 0.96776583 0.96855976 0.96923575 0.96985749 0.97052651
 0.97118486 0.97177491 0.97236967 0.97298441 0.97352346 0.97402591
 0.97458049 0.97508728 0.97560808 0.97606181 0.97649164 0.97691449
 0.97735211 0.97776941 0.97817522 0.97855329 0.97895368 0.97929253
 0.9796337  0.98000823 0.98034985 0.98065469 0.98096565 0.98129099
 0.98156097 0.9818575  0.98211601 0.98238711 0.98266321 0.98289239
 0.98312277 0.98337927 0.98361119 0.98381855 0.98401201 0.98424384
 0.98441286 0.98460649 0.98477036 0.98493632 0.98510687 0.98524454
 0.98537967 0.98553712 0.98566654 0.98577829 0.9858741  0.98599733
 0.98606441 0.98615445 0.98620891 0.98626575 0.98631142 0.98636173
 0.98637631 0.98637871 0.9863874  0.98635378 0.98631125 0.9862